# Linear regression with non-linear link between data and target

In the previous exercise, you were asked to train a linear regression model
on a dataset where the matrix `data` and the vector `target` do not have a
linear link.

In this notebook, we show that even if the parametrization of linear models
is not natively adapated to data with non-linearity, it is still possible
to make linear model more flexible and expressive.

To illustrate these concepts, we will reuse the same dataset generated in the
previous exercise.

In [ ]:
import numpy as np

rng = np.random.RandomState(0)

n_sample = 100
data_max, data_min = 1.4, -1.4
len_data = (data_max - data_min)
data = rng.rand(n_sample) * len_data - len_data / 2
sorted_idx = np.argsort(data)
noise = rng.randn(n_sample) * .3
target = data ** 3 - 0.5 * data ** 2 + noise

<div class="admonition note alert alert-info">
<p class="first admonition-title" style="font-weight: bold;">Note</p>
<p class="last">To ease the plotting, we will create a Pandas dataframe containing the data
and target</p>
</div>

In [ ]:
import pandas as pd
full_data = pd.DataFrame({"data": data, "target": target})

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context("talk")

_ = sns.scatterplot(data=full_data, x="data", y="target")

We will highlight the limitations of fitting a linear regression model as
done in the previous exercise.

<div class="admonition warning alert alert-danger">
<p class="first admonition-title" style="font-weight: bold;">Warning</p>
<p class="last">In scikit-learn, by convention <tt class="docutils literal">data</tt> (also called <tt class="docutils literal">X</tt> in the scikit-learn
documentation) should be a 2D matrix of shape <tt class="docutils literal">(n_samples, n_features)</tt>.
If <tt class="docutils literal">data</tt> is a 1D vector, you need to reshape it into a matrix with a
single column if the vector represents a feature or a single row if the
vector represents a sample.</p>
</div>

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

linear_regression = LinearRegression()
# X should be 2D for sklearn
data = data.reshape((-1, 1))
linear_regression.fit(data, target)

target_predicted = linear_regression.predict(data)
mse = mean_squared_error(target, target_predicted)

ax = sns.scatterplot(data=full_data, x="data", y="target")
ax.plot(data.ravel(), target_predicted, color="tab:orange")
_ = ax.set_title(f"Mean squared error = {mse:.2f}")

Here the coefficients learnt by `LinearRegression` is the best "straight
line" that fits the data. We can inspect the coefficients using the
attributes of the model learnt as follows:

In [ ]:
print(f"weight: {linear_regression.coef_[0]:.2f}, "
      f"intercept: {linear_regression.intercept_:.2f}")

It is important to note that the model learnt will not be able to handle the
non-linear relationship between `data` and `target` since linear models
assume the relationship between `data` and `target` to be linear.

Indeed, there are 3 possibilities to solve this issue:

1. choose a model that natively can deal with non-linearity,
2. "augment" features by including expert knowledge which can be used by
   the model, or
3. use a "kernel" to have a locally-based decision function instead of a
   global linear decision function.

Let's illustrate quickly the first point by using a decision tree regressor
which can natively handle non-linearity.

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree = DecisionTreeRegressor(max_depth=3).fit(data, target)
target_predicted = tree.predict(data)
mse = mean_squared_error(target, target_predicted)

ax = sns.scatterplot(data=full_data, x="data", y="target")
ax.plot(data[sorted_idx], target_predicted[sorted_idx],
        color="tab:orange")
_ = ax.set_title(f"Mean squared error = {mse:.2f}")

Instead of having a model which can natively deal with non-linearity, we
could also modify our data: we could create new features, derived from the
original features, using some expert knowledge. In this example, we know that
we have a cubic and squared relationship between `data` and `target` (because
we generated the data). Indeed, we could create two new features (`data ** 2`
and `data ** 3`) using this information.

In [ ]:
data = np.concatenate([data, data ** 2, data ** 3], axis=1)

linear_regression.fit(data, target)
target_predicted = linear_regression.predict(data)
mse = mean_squared_error(target, target_predicted)

ax = sns.scatterplot(data=full_data, x="data", y="target")
ax.plot(data[sorted_idx, 0], target_predicted[sorted_idx],
        color="tab:orange")
_ = ax.set_title(f"Mean squared error = {mse:.2f}")

We can see that even with a linear model, we can overcome the linearity
limitation of the model by adding the non-linear component into the design of
additional features. Here, we created new features by knowing the way the
target was generated. In practice, this is usually not the case.

Instead, one is usually creating interaction between features (e.g. $x_1
\times x_2$) with different orders (e.g. $x_1, x_1^2, x_1^3$), at the risk of
creating a model with too much flexibility where the polynomial terms allows
to fit noise in the dataset and thus lead overfit. In scikit-learn, the
`PolynomialFeatures` is a transformer to create such feature interactions
which we could have used instead of manually creating new features.

To demonstrate `PolynomialFeatures`, we are going to use a scikit-learn
pipeline which will first create the new features and then fit the model.
We come back to scikit-learn pipelines and discuss them in more detail later.

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures

data = data[:, [0]]

model = make_pipeline(PolynomialFeatures(degree=3),
                      LinearRegression())
model.fit(data, target)
target_predicted = model.predict(data)
mse = mean_squared_error(target, target_predicted)

ax = sns.scatterplot(data=full_data, x="data", y="target")
ax.plot(data[sorted_idx], target_predicted[sorted_idx], color="tab:orange")
_ = ax.set_title(f"Mean squared error = {mse:.2f}")

Thus, we saw that `PolynomialFeatures` is actually doing the same
operation that we did manually above.

The last possibility is to make a linear model more expressive is to use a
"kernel". Instead of learning a weight per feature as we previously
emphasized, a weight will be assign by sample instead. However, not all
samples will be used. This is the base of the support vector machine
algorithm.

In [ ]:
from sklearn.svm import SVR

svr = SVR(kernel="linear")
svr.fit(data, target)
target_predicted = svr.predict(data)
mse = mean_squared_error(target, target_predicted)

ax = sns.scatterplot(data=full_data, x="data", y="target")
ax.plot(data[sorted_idx], target_predicted[sorted_idx], color="tab:orange")
_ = ax.set_title(f"Mean squared error = {mse:.2f}")

The algorithm can be modified so that it can use non-linear kernel. Then,
it will compute interaction between samples using this non-linear
interaction.

In [ ]:
svr = SVR(kernel="poly", degree=3)
svr.fit(data, target)
target_predicted = svr.predict(data)
mse = mean_squared_error(target, target_predicted)

ax = sns.scatterplot(data=full_data, x="data", y="target")
ax.plot(data[sorted_idx], target_predicted[sorted_idx], color="tab:orange")
_ = ax.set_title(f"Mean squared error = {mse:.2f}")

A method supporting kernel, as SVM, allows to efficiently create a non-linear
model.